In [1]:
import json
import numpy as np

In [26]:
data = json.load(open('../result/prediction/20241019_06:52:56_sample.json'))

In [43]:
# https://www.airkorea.or.kr/web/khaiInfo?pMENU_NO=129
def apply_pm10_class(value):
    if value <= 30:
        return "좋음"
    elif value <= 80:
        return "보통"
    elif value <= 150:
        return "나쁨"
    elif value <= 600:
        return "매우나쁨"
    else:
        return ValueError
    
def apply_pm25_class(value):
    if value <= 15:
        return "좋음"
    elif value <= 35:
        return "보통"
    elif value <= 75:
        return "나쁨"
    elif value <= 500:
        return "매우나쁨"
    else:
        return ValueError
    
def apply_total_class(pm10_total_class,pm25_total_class):
    mapping = {
        "좋음": 1,
        "보통": 2,
        "나쁨": 3,
        "매우나쁨": 4
    }
    val_pm10_total_class = mapping[pm10_total_class]
    val_pm25_total_class = mapping[pm25_total_class]
    weights_sum = val_pm10_total_class*0.3+val_pm25_total_class*0.7
    if weights_sum < 1.5:
        return "좋음"
    elif weights_sum < 2.6:
        return "보통"
    elif weights_sum < 3.6:
        return "나쁨"
    else:
        return "매우나쁨"

In [47]:
data.keys()

dict_keys(['location', '1_hour', '2_hour'])

In [48]:
import copy
new_data = copy.deepcopy(data)
pm10_average_lst = []
pm25_average_lst = []
for key in new_data.keys():
    if key=='location':
        continue
    pm10_average = np.mean(np.array([i for i in new_data[key]['pm10'] if i!=0]))
    pm10_average_lst.append(pm10_average)
    pm10_class = apply_pm10_class(pm10_average)
    new_data[key]['pm10_level_per_hour'] = pm10_class
    pm25_average = np.mean(np.array([i for i in new_data[key]['pm25'] if i!=0]))
    pm25_average_lst.append(pm25_average)
    pm25_class = apply_pm25_class(pm25_average)
    new_data[key]['pm25_level_per_hour'] = pm25_class
pm10_total_class = apply_pm10_class(np.mean(pm10_average_lst))
new_data['pm10_level'] = pm10_total_class
pm25_total_class = apply_pm25_class(np.mean(pm25_average_lst))
new_data['pm25_level'] = pm25_total_class
pm_total_class = apply_total_class(pm10_total_class,pm25_total_class)
new_data['pm_total_level'] = pm_total_class

In [52]:
new_data['1_hour'].keys()

dict_keys(['pm10', 'pm25', 'pm10_level_per_hour', 'pm25_level_per_hour'])